# 1D Massive PLAXIS calculate

<div align="left" style="width: 90%"><img src="info/logofiuba.png" width="150" align="right"/><img src="info/LogoSRK.jpeg" width="195" align="right" /></div>
<br>
<font style="font-family:sans-serif" color="grey" vspace="0.1em">
Coded by<br>
<hr style="margin:0.1em; solid lightgrey" width="90%" align="left" margin="0.5em">
<ul>
    <li>N. Tasso - <a href = "mailto: ntasso@srk.com.ar">ntasso@srk.com.ar</a> - <img src="info/Version.gif" > </li></ul>

<div class="alert alert-block alert-info">
<b>Usage</b><br>
<hr style="margin:0.1em; solid lightgrey" width="90%" align="left" margin="0.5em">
<ul>
    <li>Create and calculates 1D models in Plaxis, extract the data and uploads to a Google Drive folder.</li>
</ul></div>
<div class="alert alert-block alert-warning">
<b>Input</b><br>
<hr style="margin:0.1em; solid lightgrey" width="90%" align="left" margin="0.5em">
<ul>
    <li>Statigraphy: geometry and model parameters</li>
    <li>API key of Google Drive</li>
</ul></div>
<div class="alert alert-block alert-success">   
<b>Output</b><br>
<hr style="margin:0.1em; solid lightgrey" width="90%" align="left" margin="0.5em">
<ul>
    <li>Uploads automatically</li>
</ul>
</div>
</font>

## Google drive lines

### Google drive libraries

In [ ]:
from pydrive.auth import GoogleAuth #pip install pydrive in anaconda prompt
from pydrive.drive import GoogleDrive

### Google drive functions

In [ ]:
# Move file inside Google drive
def MoveDriveFile(file_id,new_parent):
    files = drive.auth.service.files()
    file  = files.get(fileId= file_id, fields= 'parents').execute()
    prev_parents = ','.join(p['id'] for p in file.get('parents'))
    file  = files.update( fileId = file_id,
                          addParents = new_parent,
                          removeParents = prev_parents,
                          fields = 'id, parents',
                          ).execute()

### Google authorization. Json file

To upload the data to Google Drive you need to have an API key (see: https://developers.google.com/workspace/guides/create-credentials). This API key is a json file commonly named as "client_secrets.json". The json file must be saved at the same location of this code

In [ ]:
# Have you json file named as "client_secrets.json"?
gauth = GoogleAuth()           
drive = GoogleDrive(gauth)  

### Google drive folder's links

The Google Drive folder's link can be obtained from the URL of the folder. for example if the URL of the folder is: https://drive.google.com/drive/u/1/folders/1XcSLT6M9w2ODUHu7VV3gSZnj4Z1FvAo8, the link is '1XcSLT6M9w2ODUHu7VV3gSZnj4Z1FvAo8'

In [ ]:
Main_folder= '' # The main Google Drive folder of the project
Acalcular_folder='' # the Google Drive folder which contains the seismic records to be calculated
Enproceso_folder='' # the Google Drive folder which contains the seismic records that are being calculated
Terminados_folder='' # the Google Drive folder which contains the seismic records calculated
Resultados_folder='' # the Google Drive folder which collects the output data

## PLAXIS lines

### PLAXIS libraries and sync

This code was created with the PLAXIS version 21

In [ ]:
from plxscripting.easy import *

localhostport_input = 10000
localhostport_output = 10001

s_i, g_i = new_server('localhost', localhostport_input, password='')
s_o, g_o = new_server('localhost', localhostport_output, password='')


### PLAXIS functions

In [ ]:
# Create HSSSmall material
def Create_HSsmall(Name, Mat):
    g_i.soilmat(
        # --------------------- General
        'MaterialName',Name,
        'SoilModel',int(Mat[0]),
        'DrainageType',int(Mat[1]), #0 Drained, 1 Undrained.
        'gammaUnsat',Mat[2],
        'gammaSat',Mat[3],
        'RayleighAlpha',Mat[4],
        'RayleighBeta',Mat[5],
        # --------------------- Parameters
        'E50ref',Mat[6],
        'Eoedref',Mat[7],
        'EurRef', Mat[8],
        'powerm',Mat[9],
        'cref',Mat[10],
        'phi',Mat[11],
        'gamma07',Mat[12],
        'G0ref',Mat[13],

        'nu',Mat[14],
        'K0nc',Mat[15],
        #'Rf',Param[14],
        # -------------------- Groundwater
        'HydraulicModel',int(Mat[16]),
        'FlowDataModel',Mat[17],
        'perm_primary_horizontal_axis',Mat[18],
        'perm_vertical_axis',Mat[19],
        # -------------------- Initial
        'K0Determination',int(Mat[20]),
        'K0Primary',Mat[21],
        #'OCR',Param[18],
        'POP',int(Mat[22]),
    )
    
# Create PM4Sand material
def Create_PM4Sand(Name,Dr,Mat):
       g_i.soilmat(
        # -------------------- General
        'MaterialName'  , Name,
        'SoilModel'         , 100               , 
        'UserdefinedIndex'  , 0                 ,
        'UserDLLName'       , 'pm4sand64.dll'   ,
        'Usermodel'         , 'PM4Sand'         ,
        'DrainageType'      , int(Mat[0])          , #0 Drained, 1 Undrained.
        'gammaUnsat'        , Mat[1]        ,
        'gammaSat'          , Mat[2]          ,
        'RayleighAlpha'     , Mat[3]     , #From example in PLAXIS manual. VERIFY
        'RayleighBeta'      , Mat[4]      , #From example in PLAXIS manual. VERIFY
        # -------------------- Parameters
        'User1'     , Dr   , #DR0
        'User2'     , 167*np.sqrt(46*(Dr**2)+2.5)                , #G0
        'User3'     , Mat[5]               , #hp0. VER. UTILIZAN EL PROCEDIMIENTO DE BOULANGER & IDRISS (2016)
        'User4'     , Mat[6]                , #pA (Default PLAXIS manual)
        'User5'     , Mat[7]              , #emax (Default PLAXIS manual)
        'User6'     , Mat[8]              , #emin (Default PLAXIS manual)
        'User7'     , Mat[9]              , #nb (Default PLAXIS manual)
        'User8'     , Mat[10]                , #nd (Default PLAXIS manual)
        'User9'     , Mat[11]             , #phi_cv (Default PLAXIS manual)
        'User10'    , Mat[12]                , #nu (Default PLAXIS manual)
        'User11'    , Mat[13]                 , #Q (Default PLAXIS manual)
        'User12'    , Mat[14]                 , #R (Default PLAXIS manual)
        'User13'    , Mat[15]            , #PostShake (Default PLAXIS manual)
        # -------------------- Permeabilidad
        'HydraulicModel'   ,                   int(Mat[16]),
        'FlowDataModel'   ,                    Mat[17],
        'perm_primary_horizontal_axis',       Mat[18], #from galerazo. VERIFY
        'perm_vertical_axis',                 Mat[19], #from galerazo. VERIFY
        # -------------------- Interface parameters
        'Gref'              , Mat[20]              , #from previous projects. VERIFY
        'EoedRef'           , Mat[21]            , #from previous projects. VERIFY
        'UdPower'           , Mat[22]          , #from previous projects. VERIFY
        'Pref'              , Mat[23]              , #from previous projects. VERIFY
        'UDPRef'            , Mat[24]            , #from previous projects. VERIFY
        'cref'              , Mat[25]              , #from previous projects. VERIFY
        'phi'               , Mat[26]               , #from previous projects. VERIFY
        'psi'               , Mat[27]             ,  #from previous projects. VERIFY
        # ------------------- Estado inicial de tensiones
        'K0Determination',int(Mat[28]),
        'K0Primary',Mat[29],
        )
# Create linear elastic material
def Create_LE(Name,Param):
    g_i.soilmat(
        # --------------------- General
        'MaterialName',Name,
        'SoilModel',1,
        'DrainageType',0,
        'gammaUnsat',Param[0],
        'gammaSat',Param[1],
        'RayleighAlpha',Param[2],
        'RayleighBeta',Param[3],
        # --------------------- Parameters
        'nu',Param[4],
        'Eref', Param[5],
        'perm_primary_horizontal_axis',Param[6],
        'perm_vertical_axis',Param[7],
        # -------------------- Initial
        'K0Determination',0,
        'K0Primary',0.5,
    )

## Code

This code was developed to run the cases of the site-response analyses of the paper: N. Tasso, M. Sottile, A. Sfriso (2023). Seismic liquefaction potential assessment by means of automated numerical modelling. (<a href='https://www.researchgate.net/publication/371946995_Seismic_liquefaction_potential_assessment_by_means_of_automated_numerical_modelling?_sg%5B0%5D=OaDDQVy7-JBEYk2UQpv2Gr06DobCihB5bcrj-QRQGlKXkbipKKyM7hxOVSsAZ5AVvf7H9XBV45a6z80aA2uGxxSBbDci9cLUr4yFTxDn.qhBLUwt1EumXRN7L7b3vmjnG03Ol2z3ZG4KLwQl7Rq4cJa5V7h66o16OUYJ3xJVB5dK6dchTcTJsMLKED4oaOw'>ResearchGate link</a>).

<img src="info\scheme.PNG" width="300"/>

### Libraries

In [ ]:
import shutil
import os
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Local folders

In [ ]:
LocalData_folder = 'Local folders/LocalData/'
LocalSismos_folder = 'Local folders/SeismicRecords/'
LocalResultados_folder = 'Local folders/Results/'
LocalTerminados_folder = 'Local folders/Done/'
LocalBackup_folder = 'Local folders/Backup/'

### Parameters

In [ ]:
Persona = 'NT' # Who are you? Just to add to the name of the files you upload

#### Download parameters from G. Drive

In [ ]:
# list the files in the main folder
file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(Main_folder)}).GetList()

for file in file_list:

    # the Props.csv file contains the values of Dr, HC and HL that are going to be iterated
    if file['title'].endswith('Props.csv'):   
        file.GetContentFile(file['title'])
        shutil.move('Props.csv',LocalData_folder+'Props.csv')
        Props = pd.read_csv(LocalData_folder+'Props.csv', delimiter=";")

    # the NonIterativeParameters.csv contains the constant parameters (these could have been added directly in the code)
    if file['title'].endswith('NonIterativeParameters.csv'):   
        file.GetContentFile(file['title'])
        shutil.move('NonIterativeParameters.csv',LocalData_folder+'NonIterativeParameters.csv')
        NonItParam = pd.read_csv(LocalData_folder+'NonIterativeParameters.csv', delimiter=",")
    
    # the NonLiqMat.csv contains the material parameters of the non liquefiable soils
    if file['title'].endswith('NonLiqMat.csv'):   
        file.GetContentFile(file['title'])
        shutil.move('NonLiqMat.csv',LocalData_folder+'NonLiqMat.csv')
        NonLiqMat = pd.read_csv(LocalData_folder+'NonLiqMat.csv', delimiter=",")

    # the LiqMat.csv contains the material parameters of the liquefiable soil
    if file['title'].startswith('LiqMat.csv'):   
        file.GetContentFile(file['title'])
        shutil.move('LiqMat.csv',LocalData_folder+'LiqMat.csv')
        LiqMat = pd.read_csv(LocalData_folder+'LiqMat.csv', delimiter=",")

    # the RocaIntacta.csv contains the material parameters linear elastic rock at the base of the model
    if file['title'].startswith('RocaIntacta.csv'):   
        file.GetContentFile(file['title'])
        shutil.move('RocaIntacta.csv',LocalData_folder+'RocaIntacta.csv')
        RocaIntacta = pd.read_csv(LocalData_folder+'RocaIntacta.csv', delimiter=",")

#### Extract some non-iterable parameters from NonIterativeParameters.csv

In [ ]:
Htot = int(NonItParam['Parametros'][0]) # Total height of the soil column
Hrec = int(NonItParam['Parametros'][1]) # height measured from the surface were stresspoints are going to be added
g = NonItParam['Parametros'][2] # unit multiplier to change to m/s2
Sismossimultaneo = int(NonItParam['Parametros'][3]) # number of seismic to be run simultaneously
Lx = NonItParam['Parametros'][4] # width of the soil column
step = NonItParam['Parametros'][5] #separation of the stresspoints

# drainage condition
if NonItParam['Parametros'][6] == 0:
    TIPO = 'drained'
if NonItParam['Parametros'][6] ==1:
    TIPO = 'undrained'


In [ ]:
for ciclo in range(500):
    
    # Create a file list of the seismic record to be run that are inside the ToCalculate Google Drive folder
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(Acalcular_folder)}).GetList()

    # select some of the seismic records
    for j in range(min(Sismossimultaneo, len(file_list))):
        # Download the seismic record
        file_list[j].GetContentFile(file_list[j]['title'])

        # Move the seismic record to the Calculating Google Drive folder
        MoveDriveFile(file_list[j]['id'],Enproceso_folder)

        # Move the seismic record to the Calculating local folder
        shutil.move(file_list[j]['title'],LocalSismos_folder+file_list[j]['title'])
        print(file_list[j]['title'])

    # Iterate the height of the liquefiable material
    for HC in Props['H1'].dropna():

        # Iterate the height of the superficial material
        for HL in Props['H2'].dropna():

                # Create a new PLAXIS model
                s_i.new()

                # Apply project settings
                g_i.Project.UnitTime = 's'
                g_i.Project.ElementType = '15-Noded'

                # Create the materials
                Create_HSsmall('NonLiquefiable', NonLiqMat['Parametros'])
                Create_HSsmall('BelowLiquefiable', NonLiqMat['Parametros'])
                Create_LE('Bedrock',RocaIntacta['Parametros'])

                # Create the polygons
                for i in range(Htot):
                    g_i.polygon((0,-i),(Lx,-i),(Lx,-i-1),(0,-i-1))
                    if HC>i:
                        g_i.Soils[-1].Material = g_i.Materials[0]
                    else:
                        g_i.Soils[-1].Material = g_i.Materials[1]

                # A final polygon that will be the Bedrock
                g_i.polygon((0,-Htot),(Lx,-Htot),(Lx,-(Htot+Lx)),(0,-(Htot+Lx)))

                # Assign the materials to the polygons
                g_i.Soils[-1].Material = g_i.Materials[2]


                # Create interface, line displacement and load
                g_i.gotostructures()
                g_i.neginterface((Lx,-(Htot+Lx)),(0,-(Htot+Lx)))
                g_i.linedispl((Lx,-(Htot+Lx)),(0,-(Htot+Lx)))


                # Mesh the model
                g_i.gotomesh()
                for i in range(len(g_i.BoundaryLine)):
                    g_i.BoundaryLine[i].CoarsenessFactor = 0.5
                for i in range(len(g_i.Polygons)):
                    g_i.Polygons[i].CoarsenessFactor = 0.5
                g_i.mesh(0.1)

                # Add a node at the surface
                g_i.viewmesh()
                g_o.addcurvepoint("node", (Lx/2,0))

                # Add stresspoints
                for i in range(int(Hrec/step)):
                    g_o.addcurvepoint("stresspoint", (Lx/2,-i*step-0.25))
                g_o.update()

                # Add Watertable
                g_i.gotoflow()
                g_i.waterlevel((0,-HC),(Lx,-HC))

                # Go to stages tab
                g_i.gotostages()

                # Activate polygons at the initial phase
                for i in range(len(g_i.Polygons)):
                    g_i.activate(g_i.Polygons[i], g_i.Phases[0])

                # Create a nil plastic phase
                g_i.phase(g_i.Phases[0])
                g_i.Phases[1].Deform.IgnoreUndrainedBehaviour = True

                i = 0
                # For every selected seismic record
                for root, dirs, files in os.walk(LocalSismos_folder):
                    for file in files:
                        i += 1
                        if file.endswith(".csv"):

                            # Load the seismic record to Plaxis
                            DATA = np.loadtxt(LocalSismos_folder+file, delimiter="\t", skiprows=1)
                            ACEL = DATA[:,1]*g
                            TIME = DATA[:,0]
                            ACEL[0] = 0
                            dt = TIME[5]-TIME[4]
                            filename = file[:-4].split('_')[0]+'_'+str(i)
                            g_i.displmultiplier().Name = filename
                            exec('g_i.'+filename+'.Signal=1')
                            exec('g_i.'+filename+'.DataType=2')
                            A = sorted([i,j] for i,j in zip(TIME,ACEL))
                            values = '],A['.join(str(int(i)) for i in np.linspace(0,len(A)-1,len(A)))
                            values0 = 'A['+values+']'
                            exec('g_i.'+filename+'.Table.set('+values0+')')
                            exec('g_i.'+filename+'.DriftCorrection = True')
                            plt.plot(TIME,ACEL)
                            plt.show()

                            # Create a dynamic phase
                            ## Phase settings
                            g_i.phase(g_i.Phases[1])
                            g_i.Phases[-1].Identification = filename
                            g_i.Phases[-1].DeformCalcType = 'dynamic'
                            g_i.Phases[-1].Deform.TimeIntervalSeconds = int(TIME[-1])
                            g_i.Phases[-1].Deform.ResetTime = True
                            g_i.Phases[-1].Deform.UseDefaultIterationParams = False
                            m = int((len(ACEL)-1)/0.01*dt)
                            n = max(int((len(ACEL)-1)/m),1)
                            g_i.Phases[-1].Deform.MaxSteps = int(m)
                            g_i.Phases[-1].Deform.TimeStepDetermType = 'Manual'
                            g_i.Phases[-1].Deform.SubSteps = int(n)
                            g_i.Phases[-1].MaxStepsStored = 1
                            g_i.Phases[-1].MaxCores = 1

                            ## Boundary conditions
                            g_i.Dynamics.BoundaryXMin.set(g_i.Phases[-1],"Tied degrees of freedom")
                            g_i.Dynamics.BoundaryXMax.set(g_i.Phases[-1],"Tied degrees of freedom")
                            g_i.Dynamics.BoundaryYMin.set(g_i.Phases[-1],"Compliant Base")
                            g_i.GroundwaterFLow.BoundaryXMin.set(g_i.Phases[-1],"Closed")
                            g_i.GroundwaterFLow.BoundaryXMax.set(g_i.Phases[-1],"Closed")
                            g_i.Deformations.BoundaryXMin.set(g_i.Phases[-1],"Free")
                            g_i.Deformations.BoundaryXMax.set(g_i.Phases[-1],"Free")
                            g_i.Deformations.BoundaryYMin.set(g_i.Phases[-1],"Free")

                            ## Line displacement
                            g_i.LineDisplacements[0].Displacement_y.set(g_i.Phases[-1],"Fixed")
                            g_i.LineDisplacements[0].Displacement_x.set(g_i.Phases[-1],"Prescribed")
                            g_i.LineDisplacements[0].ux_start.set(g_i.Phases[-1],0.5)
                            exec('g_i.DynLineDisplacement_1_1.Multiplierx.set(g_i.Phases[-1],g_i.'+filename+')')
                            g_i.activate(g_i.DynLineDisplacement_1_1,g_i.Phases[-1])
                            g_i.activate(g_i.LineDisplacements[0],g_i.Phases[-1])

                # So far, the model is almost finished. The only thing left is to create the PM4Sand
                # material and assign to the liquefiable layer

                # For every selected relative density
                for Dr in Props['Dr'].dropna():

                    # Take time, just to measure how long it takes
                    start_time = time.time()
                    
                    # Creates the PM4Sand material based on the relative density
                    Create_PM4Sand('Liquefiable',Dr/100, LiqMat['Parametros'])

                    # Apply to the liquefiable layer in all phases
                    for i in range(int(HC),int(HC+HL),1):
                        for j in range(len(g_i.Phases)):
                            g_i.Soils[i].Material.set(g_i.Phases[j],g_i.Materials[-1])

                    # Mark for calculation all phases
                    for i in range(len(g_i.Phases)):
                        g_i.Phases[i].ShouldCalculate = True

                    # Calculate and view PLAXIS output
                    g_i.calculate()
                    g_i.view(g_i.Phases[2])

                    # For every dynamic phase
                    for Phase in g_i.Phases[2:]:

                        # Creates a dataframe with the results
                        Res = pd.DataFrame()

                        # For every selected points
                        for i in range(len(g_o.CurvePoints[:])):

                            # The first point is a node.
                            if i == 0:
                                # Extract the displacements and the acceleeration of the node
                                Res['Ux'] = np.fromstring(g_o.getcurveresultspath(g_o.CurvePoints[i],Phase,Phase,g_o.ResultTypes.Soil.Ux).echo()[1:-1], dtype=float, sep=',')
                                Res['Ax'] = np.fromstring(g_o.getcurveresultspath(g_o.CurvePoints[i],Phase,Phase,g_o.ResultTypes.Soil.Ax).echo()[1:-1], dtype=float, sep=',')
                                Res['Uy'] = np.fromstring(g_o.getcurveresultspath(g_o.CurvePoints[i],Phase,Phase,g_o.ResultTypes.Soil.Uy).echo()[1:-1], dtype=float, sep=',')
                                Res['t'] =  np.linspace(0,(len(Res['Ux'])-1)*0.01,len(Res['Ux']), endpoint=True)
                            # the remaining points are stresspoints
                            else:
                                # Extract strains, shear stress and effective vertical stress
                                Res['gamxy - Prof.'+str(i*step-0.5)] = np.fromstring(g_o.getcurveresultspath(g_o.CurvePoints[i],Phase,Phase,g_o.ResultTypes.Soil.Gamxy).echo()[1:-1], dtype=float, sep=',')
                                Res['tauxy - Prof.'+str(i*step-0.5)] = np.fromstring(g_o.getcurveresultspath(g_o.CurvePoints[i],Phase,Phase,g_o.ResultTypes.Soil.Sigxy).echo()[1:-1], dtype=float, sep=',')
                                Res['sigyyE - Prof.'+str(i*step-0.5)] = np.fromstring(g_o.getcurveresultspath(g_o.CurvePoints[i],Phase,Phase,g_o.ResultTypes.Soil.SigyyE).echo()[1:-1], dtype=float, sep=',')
                        
                        # Save the Res dataframe
                        fase = Phase.Identification.echo().split('"')[1]
                        Res.to_csv(LocalResultados_folder+fase+'HTOT'+str(Htot)+'-HL'+str(HL)+'-HC'+str(HC)+'-Dr'+str(Dr)+Persona+'-'+TIPO+'Version'+str(NonItParam['Parametros'].values[-1])+'.csv', sep=',')
                    
                    # Close the PLAXIS output
                    g_o.close()

                    # Print the time
                    print("--- %s seconds ---" % (time.time() - start_time))

    # Upload the data to Google Drive and move the local data to the Bakcup folder
    for root, dirs, files in os.walk(LocalResultados_folder):
        for file in files:
            if file.endswith(".csv"):
                shutil.move(LocalResultados_folder+file,LocalBackup_folder+file)  
                gfile = drive.CreateFile({'parents': [{'id': Resultados_folder}]})
                gfile.SetContentFile(LocalBackup_folder+file)
                gfile['title'] = file
                gfile.Upload()   

    # Move the selected seismic records to the Done local folder
    for root, dirs, files in os.walk(LocalSismos_folder):
        for file in files:
            if file.endswith(".csv"):
                shutil.move(LocalSismos_folder+file,LocalTerminados_folder+file)

    # Move the selected seismic records to the Done folder in Google Drive
    for j in range(min(Sismossimultaneo, len(file_list))):
        MoveDriveFile(file_list[j]['id'],Terminados_folder)
